# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
units = "imperial"

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

632

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [5]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
name = []
lng = []
lat = []
maxtemp = []
humid = []
cloud = []
windspd = []
country = []
datetime = []

city_number = 0

for city in cities:
    q_url = f"{base_url}q={city}&units={units}&appid={weather_api_key}"
    weather_resp = requests.get(q_url).json()

    try:
        city_number += 1
        
        name.append(weather_resp['name'])
        lng.append(weather_resp['coord']['lon'])
        lat.append(weather_resp['coord']['lat'])
        maxtemp.append(weather_resp['main']['temp_max'])
        humid.append(weather_resp['main']['humidity'])
        cloud.append(weather_resp['clouds']['all'])
        windspd.append(weather_resp['wind']['speed'])
        country.append(weather_resp['sys']['country'])
        datetime.append(weather_resp['dt'])
        

        print(f"Processing Record {city_number} of {len(cities)}: {city}")


    except (KeyError, IndexError):
        print("City not found. Skipping...")
        pass

iki
Processing Record 110 of 632: ouesso
Processing Record 111 of 632: coruripe
Processing Record 112 of 632: vila franca do campo
Processing Record 113 of 632: pottsville
Processing Record 114 of 632: kharp
Processing Record 115 of 632: batticaloa
City not found. Skipping...
Processing Record 117 of 632: nerl
Processing Record 118 of 632: geraldton
Processing Record 119 of 632: norman wells
Processing Record 120 of 632: myre
Processing Record 121 of 632: cape town
Processing Record 122 of 632: namibe
Processing Record 123 of 632: bethel
Processing Record 124 of 632: buzmeyin
Processing Record 125 of 632: oranjemund
City not found. Skipping...
Processing Record 127 of 632: horki
Processing Record 128 of 632: hirara
Processing Record 129 of 632: grindavik
Processing Record 130 of 632: praia da vitoria
Processing Record 131 of 632: saint-augustin
Processing Record 132 of 632: katsuura
Processing Record 133 of 632: pangai
Processing Record 134 of 632: te anau
Processing Record 135 of 632:

In [6]:
city_summary = pd.DataFrame({"City Name": name,
                            "Longitude": lng,
                            "Latitude": lat,
                            "Max Temperature": maxtemp,
                            "Humidtiy": humid,
                            "Cloud Coverage": cloud,
                            "Wind Speed": windspd,
                            "City Country": country,
                            "Date Time": datetime})
city_summary.head()

,City Name,Longitude,Latitude,Max Temperature,Humidtiy,Cloud Coverage,Wind Speed,City Country,Date Time
0,Hermanus,19.23,-34.42,50.00,91,100,11.32,ZA,1594342872
1,Khatanga,102.50,71.97,56.19,55,0,18.70,RU,1594342873
2,Iqaluit,-68.51,63.75,48.20,76,75,4.70,CA,1594342752
3,Yellowknife,-114.35,62.46,68.00,48,75,9.17,CA,1594342542
4,Katherine,132.27,-14.47,78.80,54,1,9.17,AU,1594342873


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [7]:
city_summary.to_csv("city_summary.csv")

In [9]:
city_sum_path = "city_summary.csv"
city_sum_csv = pd.read_csv(city_sum_path)
city_sum_csv

,Unnamed: 0,City Name,Longitude,Latitude,Max Temperature,Humidtiy,Cloud Coverage,Wind Speed,City Country,Date Time
0,0,Hermanus,19.23,-34.42,50.00,91,100,11.32,ZA,1594342872
1,1,Khatanga,102.50,71.97,56.19,55,0,18.70,RU,1594342873
2,2,Iqaluit,-68.51,63.75,48.20,76,75,4.70,CA,1594342752
3,3,Yellowknife,-114.35,62.46,68.00,48,75,9.17,CA,1594342542
4,4,Katherine,132.27,-14.47,78.80,54,1,9.17,AU,1594342873
...,...,...,...,...,...,...,...,...,...,...
572,572,Nishihara,127.76,26.18,87.80,74,75,17.22,JP,1594343022
573,573,Murdochville,-65.50,48.96,58.50,93,0,2.48,CA,1594343022
574,574,City of San Pedro,121.02,14.35,91.00,74,40,4.70,PH,1594343022
575,575,Lhokseumawe,97.15,5.18,78.44,79,99,4.05,ID,1594343022


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression